#DATA

In [ ]:
import glob

In [ ]:
import glob 
import numpy as np # Define the path to the main folder 
main_folder = '/content/drive/MyDrive/DGM/ShapeNet_pointclouds/02691156'
# Get the names of all subfolders in the main folder 
subfolders = glob.glob(main_folder + '/*/') 
# Initialize the NumPy array to store the point clouds 
point_clouds = [] 
# Loop over the subfolders 
for folder in subfolders: 
# Get the name of the pointcloud_1024.npy file in the subfolder 
  file = folder + '/pointcloud_1024.npy'
  # Load the .npy file and append it to the NumPy array 
  point_cloud = np.load(file) 
  point_clouds.append(point_cloud) 
# Convert the list of point clouds to a NumPy array 
point_clouds = np.array(point_clouds)

In [ ]:
import numpy as np


In [ ]:
point_clouds = np.load("/content/drive/MyDrive/DGM/ShapeNet_pointclouds/train.npy")

In [ ]:
len(point_clouds)

4032

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader


# Convert the NumPy array to a PyTorch tensor
point_clouds = torch.from_numpy(point_clouds[:4032])

# Create a TensorDataset from the tensor
dataset = TensorDataset(point_clouds)

# Create a DataLoader from the dataset
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
point_clouds = torch.from_numpy(point_clouds[:4032])

In [ ]:
dataset = TensorDataset(normalized)

# Create a DataLoader from the dataset
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
asdf=np.asarray(point_clouds)

In [ ]:
np.save('train.npy', asdf)

In [ ]:
f=np.load('train.npy') 

In [ ]:
f.shape

(4032, 1024, 3)

#Model

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

# Define the model
class PointCloudAutoEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    
    # Encoder layers
    self.conv1 = nn.Conv1d(3, 16, kernel_size=3, stride=2, padding=1)
    self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1)
    self.conv3 = nn.Conv1d(32, 8, kernel_size=3, stride=2, padding=1)
    
    # Latent space
    self.fc1 = nn.Linear(1024, 128)
    
    # Decoder layers
    self.fc2 = nn.Linear(128, 1024)
    self.deconv1 = nn.ConvTranspose1d(64, 32, kernel_size=4, stride=4)
    self.deconv2 = nn.ConvTranspose1d(32, 16,kernel_size=4, stride=4)
    self.deconv3 = nn.ConvTranspose1d(16, 3, kernel_size=4, stride=4)
    
  def forward(self, x):
  # Encoder
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    
    # Latent space
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    
    # Decoder
    x = self.fc2(x)
    x = x.view(x.size(0), 64, 16)
    x = self.deconv1(x)
    x = self.deconv2(x)
    x = self.deconv3(x)
    
    return x



#Training

In [ ]:
num_epochs=100

In [ ]:
import torch
from torch import optim

# Define the model and move it to the GPU
model = PointCloudAutoEncoder()
if torch.cuda.is_available():
    model = model.cuda()

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.MSELoss()


# Train the model
for epoch in range(num_epochs):
  for data in dataloader:
      # Move the data to the GPU
      if torch.cuda.is_available():
          data = data[0].reshape(-1,3,1024).to("cuda", dtype=torch.float)

      # Forward pass
      output = model(data)

      # Calculate the loss
      loss = loss_fn(output, data)
      print(loss)
      # Backward pass and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()


In [ ]:
x=point_clouds[1000].reshape(-1,3,1024)
x.shape

torch.Size([1, 3, 1024])

In [ ]:
x=x.to("cuda", dtype=torch.float)
output = model(x)

In [ ]:
z=np.asarray(output.reshape(1024,3).to("cpu").detach())

In [ ]:
len(z)

1024

In [ ]:
np.save('r2.npy', z)

In [ ]:
model.conv1

Conv1d(3, 16, kernel_size=(3,), stride=(2,), padding=(1,))

In [ ]:
import pandas as pd

In [ ]:
z=point_clouds[1000]

In [ ]:
df = pd.DataFrame(z, columns = ['A','B','C'])


In [ ]:
z.shape

(1024, 3)

In [ ]:
df

,A,B,C
0,"tensor(-0.0029, dtype=torch.float64)","tensor(0.0230, dtype=torch.float64)","tensor(0.0137, dtype=torch.float64)"
1,"tensor(0.0057, dtype=torch.float64)","tensor(0.0037, dtype=torch.float64)","tensor(0.4825, dtype=torch.float64)"
2,"tensor(0.0060, dtype=torch.float64)","tensor(-0.1527, dtype=torch.float64)","tensor(-0.2984, dtype=torch.float64)"
3,"tensor(0.0057, dtype=torch.float64)","tensor(0.1571, dtype=torch.float64)","tensor(-0.2750, dtype=torch.float64)"
4,"tensor(-0.0119, dtype=torch.float64)","tensor(-0.2798, dtype=torch.float64)","tensor(0.0319, dtype=torch.float64)"
...,...,...,...
1019,"tensor(0.0298, dtype=torch.float64)","tensor(-0.0623, dtype=torch.float64)","tensor(-0.0842, dtype=torch.float64)"
1020,"tensor(-0.0194, dtype=torch.float64)","tensor(-0.0734, dtype=torch.float64)","tensor(0.0311, dtype=torch.float64)"
1021,"tensor(-0.0444, dtype=torch.float64)","tensor(-0.0176, dtype=torch.float64)","tensor(0.0612, dtype=torch.float64)"
1022,"tensor(0.0061, dtype=torch.float64)","tensor(-0.1220, dtype=torch.float64)","tensor(-0.2405, dtype=torch.float64)"


In [ ]:
import plotly.express as px
fig = px.scatter_3d(df ,x='A', y='B', z='C')
fig.show()


In [ ]:
data=point_clouds

In [ ]:
import torch

In [ ]:
mean = torch.mean(data, dim=1, keepdim=True)

# Subtract the mean from each point
normalized = data - mean


In [ ]:
for i in range(len(data)):
  data[i]=torch.Tensor(data[i])

In [ ]:
type(torch.Tensor(data[0]))

torch.Tensor

In [ ]:
import torch

# Generate dummy data with the shape [batch_size, 1024, 3]
data = torch.randn(32, 1024, 3)

# Calculate the mean of the point coordinates
mean = torch.mean(data, dim=1, keepdim=True)

# Subtract the mean from each point
output = data - mean

# Print the shape of the output tensor
print(output.shape)

torch.Size([32, 1024, 3])


#projection loss

In [ ]:
def create_projection(point_cloud):
    # Generate the x, y, and z coordinates of the points
    x = point_cloud[:, 0]
    y = point_cloud[:, 1]
    z = point_cloud[:, 2]

    # Calculate the 2D projection of the points
    projection = torch.stack([x, y], dim=1)

    return projection

# Generate a batch of dummy point cloud data with the shape [batch_size, num_points, 3]
generated = torch.randn(32, 1024, 3)
ground_truth = torch.randn(32, 1024, 3)

# Create 2D projections of the point clouds
generated_projection = create_projection(generated)
ground_truth_projection = create_projection(ground_truth)

# Calculate the loss between the generated and ground truth projections
loss = torch.nn.MSELoss()(generated_projection, ground_truth_projection)

# Print the calculated loss
print(loss)

tensor(2.0317)


In [ ]:
# Define the loss function
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())

# Train the model for a number of epochs
for epoch in range(num_epochs):
    # Loop over the point cloud data
    for point_cloud, ground_truth in data_loader:
        # Create projections of the point cloud and ground truth
        projections = []
        for viewpoint in viewpoints:
            projections.append(create_projection(point_cloud, viewpoint))
            projections.append(create_projection(ground_truth, viewpoint))

        # Concatenate the projections
        projections = torch.cat(projections, dim=1)

        # Move the data to the GPU, if available
        if torch.cuda.is_available():
            point_cloud = point_cloud.cuda()
            ground_truth = ground_truth.cuda()
            projections = projections.cuda()

        # Pass the data through the model
        decoded = model(point_cloud)

        # Calculate the reconstruction loss
        reconstruction_loss = loss_fn(decoded, ground_truth)

        # Calculate the projection loss
        projection_loss = loss_fn(projections, decoded)

        # Calculate the total loss
        loss = reconstruction_loss + projection_loss

        # Backpropagate the error and update the model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for the current epoch
    print(f"Epoch {epoch}: Loss = {loss.item()}")